<a href="https://colab.research.google.com/github/tsemanfang628-prog/Git_series/blob/main/ai_recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install ipywidgets matplotlib pillow faker
!pip install requests

import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import json
from datetime import datetime
from faker import Faker
import random

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.8 MB/s eta 0:00:00


In [2]:
class RecipeGenerator:
    def __init__(self):
        self.faker = Faker()
        self.setup_widgets()
        self.setup_layout()

    def setup_widgets(self):
        """Create interactive widgets for recipe parameters"""

        # Main ingredients
        self.main_ingredient = widgets.Dropdown(
            options=['chicken', 'beef', 'fish', 'pasta', 'rice', 'vegetables', 'tofu', 'eggs'],
            value='chicken',
            description='🍗 Main Ingredient:',
            style={'description_width': '120px'}
        )

        # Cuisine type
        self.cuisine = widgets.Dropdown(
            options=['Italian', 'Chinese', 'Mexican', 'Indian', 'Japanese', 'American', 'Mediterranean', 'Thai'],
            value='Italian',
            description='🌍 Cuisine:',
            style={'description_width': '120px'}
        )

        # Eating occasion
        self.occasion = widgets.Dropdown(
            options=['breakfast', 'lunch', 'dinner', 'snack', 'dessert', 'party', 'healthy', 'quick meal'],
            value='dinner',
            description='⏰ Occasion:',
            style={'description_width': '120px'}
        )

        # Cooking time
        self.cooking_time = widgets.Dropdown(
            options=['15 minutes', '30 minutes', '45 minutes', '1 hour', '1.5 hours', '2+ hours'],
            value='30 minutes',
            description='⏱️ Time:',
            style={'description_width': '120px'}
        )

        # Dietary restrictions
        self.diet = widgets.SelectMultiple(
            options=['vegetarian', 'vegan', 'gluten-free', 'dairy-free', 'low-carb', 'high-protein', 'keto'],
            value=[],
            description='🥗 Diet:',
            rows=3,
            style={'description_width': '120px'}
        )

        # Spice level
        self.spice_level = widgets.Dropdown(
            options=['mild', 'medium', 'spicy', 'very spicy'],
            value='medium',
            description='🌶️ Spice:',
            style={'description_width': '120px'}
        )

        # Generate button
        self.generate_btn = widgets.Button(
            description='👨‍🍳 GENERATE RECIPE & PHOTO',
            button_style='success',
            layout=widgets.Layout(width='250px', height='40px')
        )

        # Export button
        self.export_btn = widgets.Button(
            description='💾 EXPORT TO GITHUB',
            button_style='info',
            layout=widgets.Layout(width='200px', height='40px')
        )

        # Output area
        self.output = widgets.Output()

        # Connect buttons
        self.generate_btn.on_click(self.generate_recipe)
        self.export_btn.on_click(self.export_to_github)

    def setup_layout(self):
        """Setup the layout"""
        left_panel = widgets.VBox([
            self.main_ingredient,
            self.cuisine,
            self.occasion
        ])

        right_panel = widgets.VBox([
            self.cooking_time,
            self.spice_level,
            self.diet
        ])

        self.main_layout = widgets.VBox([
            widgets.HTML("<h1 style='color: #e74c3c; text-align: center;'>🍳 AI RECIPE GENERATOR</h1>"),
            widgets.HTML("<h3 style='text-align: center;'>Create custom recipes with AI-generated photos</h3>"),

            widgets.HBox([left_panel, right_panel]),

            widgets.HTML("<br>"),

            widgets.HBox([
                self.generate_btn,
                self.export_btn
            ], layout=widgets.Layout(justify_content='center')),

            widgets.HTML("<br>"),

            self.output
        ])

    def display(self):
        """Display the GUI"""
        display(self.main_layout)

    def generate_recipe(self, b):
        """Generate recipe and food photo"""
        with self.output:
            clear_output()
            print("👨‍🍳 Generating your custom recipe...")

            # Get parameters
            params = {
                'main_ingredient': self.main_ingredient.value,
                'cuisine': self.cuisine.value,
                'occasion': self.occasion.value,
                'cooking_time': self.cooking_time.value,
                'diet': list(self.diet.value),
                'spice_level': self.spice_level.value
            }

            # Generate recipe
            recipe = self.create_recipe(params)

            # Generate food photo
            self.create_food_photo(recipe, params)

            # Store for export
            self.current_recipe = recipe
            self.current_params = params

            print("✅ Recipe generated successfully!")

    def create_recipe(self, params):
        """Create a custom recipe based on parameters"""

        # Recipe templates
        cuisine_templates = {
            'Italian': {
                'dishes': ['Pasta', 'Risotto', 'Pizza', 'Bruschetta'],
                'herbs': ['basil', 'oregano', 'rosemary', 'thyme'],
                'base': ['tomato sauce', 'olive oil', 'garlic', 'parmesan']
            },
            'Chinese': {
                'dishes': ['Stir-fry', 'Fried Rice', 'Noodles', 'Dumplings'],
                'herbs': ['ginger', 'garlic', 'scallions', 'soy sauce'],
                'base': ['soy sauce', 'sesame oil', 'rice vinegar', 'oyster sauce']
            },
            'Mexican': {
                'dishes': ['Tacos', 'Burritos', 'Quesadillas', 'Enchiladas'],
                'herbs': ['cilantro', 'cumin', 'chili powder', 'lime'],
                'base': ['beans', 'cheese', 'salsa', 'avocado']
            },
            'Indian': {
                'dishes': ['Curry', 'Biryani', 'Tikka Masala', 'Samosa'],
                'herbs': ['turmeric', 'cumin', 'coriander', 'garam masala'],
                'base': ['yogurt', 'coconut milk', 'onions', 'tomatoes']
            }
        }

        template = cuisine_templates.get(params['cuisine'], cuisine_templates['Italian'])

        # Generate recipe name
        dish_name = f"{params['main_ingredient'].title()} {random.choice(template['dishes'])}"

        # Generate ingredients
        ingredients = [
            f"500g {params['main_ingredient']}",
            f"2 tbsp {random.choice(template['base'])}",
            f"1 tbsp {random.choice(template['herbs'])}",
            "1 onion, chopped",
            "2 cloves garlic, minced",
            "Salt and pepper to taste"
        ]

        # Add diet-specific ingredients
        if 'vegetarian' in params['diet'] and params['main_ingredient'] in ['chicken', 'beef', 'fish']:
            ingredients[0] = "500g tofu or tempeh"
        if 'gluten-free' in params['diet'] and 'pasta' in dish_name.lower():
            ingredients[0] = "500g gluten-free pasta"

        # Generate instructions
        instructions = [
            f"1. Prepare all ingredients according to {params['cuisine']} style",
            f"2. Cook the {params['main_ingredient']} for optimal flavor",
            f"3. Add herbs and spices for {params['spice_level']} taste",
            f"4. Combine all ingredients and cook for {params['cooking_time']}",
            f"5. Serve as a perfect {params['occasion']} meal"
        ]

        # Nutrition info
        nutrition = {
            'calories': random.randint(300, 800),
            'protein': f"{random.randint(20, 40)}g",
            'carbs': f"{random.randint(30, 80)}g",
            'fat': f"{random.randint(10, 30)}g"
        }

        recipe = {
            'name': dish_name,
            'cuisine': params['cuisine'],
            'occasion': params['occasion'],
            'cooking_time': params['cooking_time'],
            'spice_level': params['spice_level'],
            'dietary': params['diet'],
            'ingredients': ingredients,
            'instructions': instructions,
            'nutrition': nutrition,
            'recipe_id': self.faker.uuid4(),
            'created_at': datetime.now().isoformat()
        }

        # Display recipe
        self.display_recipe(recipe)

        return recipe

    def display_recipe(self, recipe):
        """Display the generated recipe"""
        print(f"\n🎯 RECIPE: {recipe['name']}")
        print("=" * 50)

        print(f"🌍 Cuisine: {recipe['cuisine']}")
        print(f"⏰ Occasion: {recipe['occasion']}")
        print(f"⏱️ Cooking Time: {recipe['cooking_time']}")
        print(f"🌶️ Spice Level: {recipe['spice_level']}")
        print(f"🥗 Dietary: {', '.join(recipe['dietary']) if recipe['dietary'] else 'None'}")

        print(f"\n📋 INGREDIENTS:")
        for ingredient in recipe['ingredients']:
            print(f"  • {ingredient}")

        print(f"\n👨‍🍳 INSTRUCTIONS:")
        for step in recipe['instructions']:
            print(f"  {step}")

        print(f"\n📊 NUTRITION (per serving):")
        print(f"  Calories: {recipe['nutrition']['calories']} kcal")
        print(f"  Protein: {recipe['nutrition']['protein']}")
        print(f"  Carbs: {recipe['nutrition']['carbs']}")
        print(f"  Fat: {recipe['nutrition']['fat']}")

    def create_food_photo(self, recipe, params):
        """Create a food photo visualization"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Left: Food illustration
        colors = {
            'chicken': '#f39c12', 'beef': '#c0392b', 'fish': '#3498db',
            'pasta': '#f1c40f', 'rice': '#ecf0f1', 'vegetables': '#27ae60',
            'tofu': '#bdc3c7', 'eggs': '#f1c40f'
        }

        main_color = colors.get(params['main_ingredient'], '#e74c3c')

        # Create simple food illustration
        if params['main_ingredient'] in ['chicken', 'beef']:
            # Meat dish
            ax1.add_patch(plt.Circle((0.5, 0.6), 0.3, color=main_color, alpha=0.8))
            ax1.add_patch(plt.Rectangle((0.3, 0.3), 0.4, 0.2, color='#27ae60', alpha=0.7))  # veggies
        elif params['main_ingredient'] == 'pasta':
            # Pasta dish
            for i in range(10):
                x = random.uniform(0.2, 0.8)
                y = random.uniform(0.3, 0.8)
                ax1.add_patch(plt.Circle((x, y), 0.05, color=main_color, alpha=0.8))
        else:
            # General dish
            ax1.add_patch(plt.Circle((0.5, 0.5), 0.4, color=main_color, alpha=0.7))

        ax1.set_xlim(0, 1)
        ax1.set_ylim(0, 1)
        ax1.set_title('🍽️ Your Dish', fontsize=16, fontweight='bold')
        ax1.axis('off')

        # Right: Recipe info graphic
        info_text = f"""
        {recipe['name']}

        ⏱️ {recipe['cooking_time']}
        🌶️ {recipe['spice_level'].title()}
        🌍 {recipe['cuisine']}
        ⏰ {recipe['occasion'].title()}

        🥗 Dietary:
        {', '.join(recipe['dietary']) if recipe['dietary'] else 'Standard'}

        📊 Nutrition:
        {recipe['nutrition']['calories']} kcal
        {recipe['nutrition']['protein']} protein
        """

        ax2.text(0.1, 0.9, info_text, fontsize=12, verticalalignment='top',
                linespacing=1.8, fontfamily='sans-serif')
        ax2.set_xlim(0, 1)
        ax2.set_ylim(0, 1)
        ax2.set_title('📝 Recipe Details', fontsize=16, fontweight='bold')
        ax2.axis('off')

        plt.suptitle(f"👨‍🍳 AI-GENERATED RECIPE: {recipe['name']}",
                    fontsize=18, fontweight='bold', y=0.95)
        plt.tight_layout()
        plt.show()

        # Save the image
        plt.savefig(f"recipe_{recipe['name'].replace(' ', '_')}.png",
                   dpi=150, bbox_inches='tight', facecolor='white')

    def export_to_github(self, b):
        """Export recipe to GitHub-ready format"""
        with self.output:
            clear_output()

            if not hasattr(self, 'current_recipe'):
                print("❌ Please generate a recipe first!")
                return

            print("💾 Preparing files for GitHub...")

            # Create filename
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            recipe_name_clean = self.current_recipe['name'].replace(' ', '_').lower()

            # 1. Save recipe as JSON
            json_filename = f"{recipe_name_clean}_{timestamp}.json"
            with open(json_filename, 'w') as f:
                json.dump({
                    'recipe': self.current_recipe,
                    'parameters': self.current_params,
                    'exported_at': datetime.now().isoformat()
                }, f, indent=2)

            # 2. Save recipe as Markdown (for GitHub README)
            md_filename = f"{recipe_name_clean}_{timestamp}.md"
            self.create_markdown_file(md_filename)

            # 3. Create requirements file for GitHub
            self.create_requirements_file()

            # 4. Create main Python script
            self.create_main_script()

            print("✅ Files created for GitHub:")
            print(f"   📄 {json_filename} (Recipe data)")
            print(f"   📄 {md_filename} (Documentation)")
            print(f"   📄 requirements.txt (Dependencies)")
            print(f"   📄 recipe_generator.py (Main script)")
            print(f"   📄 recipe_{recipe_name_clean}.png (Food photo)")

            print("\n🎯 Next steps to connect to GitHub:")
            print("1. Go to https://github.com and create new repository")
            print("2. Upload these files to your repository")
            print("3. Your recipe generator is now on GitHub!")

    def create_markdown_file(self, filename):
        """Create a Markdown file for GitHub"""
        md_content = f"""# {self.current_recipe['name']}

![Food Photo](recipe_{self.current_recipe['name'].replace(' ', '_')}.png)

## Recipe Details

- **Cuisine**: {self.current_recipe['cuisine']}
- **Occasion**: {self.current_recipe['occasion']}
- **Cooking Time**: {self.current_recipe['cooking_time']}
- **Spice Level**: {self.current_recipe['spice_level']}
- **Dietary**: {', '.join(self.current_recipe['dietary']) if self.current_recipe['dietary'] else 'None'}

## Ingredients

{chr(10).join(['- ' + ingredient for ingredient in self.current_recipe['ingredients']])}

## Instructions

{chr(10).join(self.current_recipe['instructions'])}

## Nutrition

- Calories: {self.current_recipe['nutrition']['calories']} kcal
- Protein: {self.current_recipe['nutrition']['protein']}
- Carbohydrates: {self.current_recipe['nutrition']['carbs']}
- Fat: {self.current_recipe['nutrition']['fat']}

## Generated By

AI Recipe Generator - Created with Google Colab
"""

        with open(filename, 'w') as f:
            f.write(md_content)

    def create_requirements_file(self):
        """Create requirements.txt for GitHub"""
        requirements = """ipywidgets>=7.0.0
matplotlib>=3.0.0
pillow>=8.0.0
faker>=8.0.0
numpy>=1.0.0
"""
        with open('requirements.txt', 'w') as f:
            f.write(requirements)

    def create_main_script(self):
        """Create main Python script for GitHub"""
        script_content = '''"""
AI Recipe Generator
Automatically generates recipes and food photos based on user preferences
"""

import json
import random
from datetime import datetime

class SimpleRecipeGenerator:
    def __init__(self):
        self.recipes_generated = 0

    def generate_recipe(self, main_ingredient, cuisine, occasion):
        """Generate a simple recipe"""
        # This is a simplified version for GitHub
        recipe = {
            'name': f"{main_ingredient.title()} {cuisine} Dish",
            'ingredients': [
                f"500g {main_ingredient}",
                "2 tbsp cooking oil",
                "1 onion, chopped",
                "2 cloves garlic",
                "Salt and pepper to taste"
            ],
            'instructions': [
                "1. Prepare all ingredients",
                "2. Cook according to preferences",
                "3. Serve and enjoy!"
            ]
        }
        return recipe

# Example usage
if __name__ == "__main__":
    generator = SimpleRecipeGenerator()
    recipe = generator.generate_recipe("chicken", "Italian", "dinner")
    print("Generated Recipe:", recipe['name'])
'''
        with open('recipe_generator.py', 'w') as f:
            f.write(script_content)

# Create and display the recipe generator
print("=== 🍳 AI RECIPE GENERATOR ===")
print("Simple, Google Colab compatible, GitHub ready!")
recipe_gen = RecipeGenerator()
recipe_gen.display()

=== 🍳 AI RECIPE GENERATOR ===
Simple, Google Colab compatible, GitHub ready!


In [3]:
# Add quick recipe templates
def add_quick_templates(recipe_gen):
    """Add quick template buttons"""

    templates = {
        '🍝 Quick Pasta': {
            'main_ingredient': 'pasta',
            'cuisine': 'Italian',
            'occasion': 'dinner',
            'cooking_time': '20 minutes',
            'spice_level': 'mild'
        },
        '🌯 Healthy Bowl': {
            'main_ingredient': 'vegetables',
            'cuisine': 'Mediterranean',
            'occasion': 'lunch',
            'cooking_time': '15 minutes',
            'spice_level': 'medium'
        },
        '🍗 Protein Dinner': {
            'main_ingredient': 'chicken',
            'cuisine': 'American',
            'occasion': 'dinner',
            'cooking_time': '30 minutes',
            'spice_level': 'medium'
        }
    }

    buttons = []
    for name, params in templates.items():
        btn = widgets.Button(
            description=name,
            button_style='primary',
            layout=widgets.Layout(width='180px', margin='5px')
        )

        def create_handler(template_params):
            def handler(b):
                recipe_gen.main_ingredient.value = template_params['main_ingredient']
                recipe_gen.cuisine.value = template_params['cuisine']
                recipe_gen.occasion.value = template_params['occasion']
                recipe_gen.cooking_time.value = template_params['cooking_time']
                recipe_gen.spice_level.value = template_params['spice_level']
                recipe_gen.diet.value = []

                with recipe_gen.output:
                    clear_output()
                    print(f"✅ Loaded: {b.description}")
            return handler

        btn.on_click(create_handler(params))
        buttons.append(btn)

    template_box = widgets.HBox(buttons, layout=widgets.Layout(justify_content='center'))

    # Add to main layout
    recipe_gen.main_layout.children = (recipe_gen.main_layout.children[0],
                                      recipe_gen.main_layout.children[1],
                                      template_box,
                                      *recipe_gen.main_layout.children[2:])

# Add templates to our generator
add_quick_templates(recipe_gen)

print("\n🎯 QUICK START TEMPLATES ADDED!")
print("Click any template above to get started quickly!")


🎯 QUICK START TEMPLATES ADDED!
Click any template above to get started quickly!


In [6]:
# Display GitHub setup instructions
print("""
🚀 GITHUB INTEGRATION GUIDE
===========================

AFTER GENERATING RECIPES:

1. 📁 CREATE GITHUB REPOSITORY:
   - Go to https://github.com
   - Click '+' → 'New repository'
   - Name: 'ai-recipe-generator'
   - Add README: Yes
   - Create repository

2. 💾 UPLOAD FILES:
   - Click 'Add file' → 'Upload files'
   - Upload all generated files:
     * .json files (recipe data)
     * .md files (documentation)
     * .png files (food photos)
     * requirements.txt
     * recipe_generator.py

3. 🔧 SETUP COLAB-GITHUB CONNECTION:
   In Google Colab, run:
4. 🎯 CONTINUOUS UPDATES:
- Generate new recipes in Colab
- Upload new files to GitHub
- Keep your recipe collection growing!

Your AI Recipe Generator is now GitHub-ready! 🍳
""")


🚀 GITHUB INTEGRATION GUIDE

AFTER GENERATING RECIPES:

1. 📁 CREATE GITHUB REPOSITORY:
   - Go to https://github.com
   - Click '+' → 'New repository'
   - Name: 'ai-recipe-generator'
   - Add README: Yes
   - Create repository

2. 💾 UPLOAD FILES:
   - Click 'Add file' → 'Upload files'
   - Upload all generated files:
     * .json files (recipe data)
     * .md files (documentation) 
     * .png files (food photos)
     * requirements.txt
     * recipe_generator.py

3. 🔧 SETUP COLAB-GITHUB CONNECTION:
   In Google Colab, run:
4. 🎯 CONTINUOUS UPDATES:
- Generate new recipes in Colab
- Upload new files to GitHub
- Keep your recipe collection growing!

Your AI Recipe Generator is now GitHub-ready! 🍳



In [7]:
# Final display
print("""
✅ AI RECIPE GENERATOR - READY!
================================

FEATURES:
🍳 Generate custom recipes based on ingredients & occasion
📸 Create food photos and nutrition info
💾 Export to GitHub with one click
📁 Simple Colab-compatible code
🎯 Quick templates for fast start

HOW TO USE:
1. Choose your parameters (ingredient, cuisine, occasion, etc.)
2. Click 'GENERATE RECIPE & PHOTO'
3. View your custom recipe with food illustration
4. Click 'EXPORT TO GITHUB' to save everything

Start by choosing parameters or clicking a quick template! 👇
""")

# Force refresh display
display(recipe_gen.main_layout)


✅ AI RECIPE GENERATOR - READY!

FEATURES:
🍳 Generate custom recipes based on ingredients & occasion
📸 Create food photos and nutrition info  
💾 Export to GitHub with one click
📁 Simple Colab-compatible code
🎯 Quick templates for fast start

HOW TO USE:
1. Choose your parameters (ingredient, cuisine, occasion, etc.)
2. Click 'GENERATE RECIPE & PHOTO' 
3. View your custom recipe with food illustration
4. Click 'EXPORT TO GITHUB' to save everything

Start by choosing parameters or clicking a quick template! 👇

